# Simple Machine Learning model using Overture Maps data

In this notebook, we aim to predict the distance to the nearest bicycle-sharing station in three Spanish cities: Madrid, Seville, and Valencia.

Our goal is to develop a model capable of accurately predicting this distance based on geospatial features, which can assist in urban planning and enhance the accessibility of bicycle-sharing services.

The task involves several key steps:

1. **Loading bicycle-sharing stations**: 
We begin by loading geospatial data for bicycle-sharing stations from OpenStreetMap using [`OsmOnlineLoader`](../../loaders/osm_online_loader/) for the specified cities. We also visualise the locations of these stations.

2. **Regionalization using H3 hexagons**:
The cities are regionalized using H3 hexagons with [`H3Regionalizer`](../../regionalizers/h3_regionalizer/), enabling us to divide them into smaller, manageable regions. Next, we buffer the generated H3 regions and compute the distance from each hexagon to the nearest bicycle-sharing station.

3. **Feature extraction and embedding**:
We load additional geospatial features from Overture Maps using [`OvertureMapsLoader`](../../loaders/overture_maps_loader/) and join them with the H3 regions (with [`IntersectionJoiner`](../../joiners/intersection_joiner/)). We then generate embeddings for these regions using [`ContextualCountEmbedder`](../../embedders/contextual_count_embedder/), which will be used as input features for our machine learning model.

4. **Training and evaluating the model**: 
We prepare the data for training and validation, standardize the features, and train an XGBoost regression model to predict the distance to the nearest bicycle-sharing station. Data from Madrid is used for training, while data from Seville is used for validation.

5. **Prediction and Visualization**:
Finally, the trained model predicts the distance to the nearest bicycle-sharing station for all three cities. The predicted distances and prediction errors are visualised on maps to evaluate the model's performance.

<div class="admonition info">
    <p class="admonition-title">Prerequisites</p>
    <p>
    <ul>
        <li>12 GB of RAM</li>
        <li>
            Installed libraries: 
            <code>srai[osm,overturemaps,plotting]</code>, 
            <code>contextily</code>, <code>seaborn</code>, 
            <code>scikit-learn</code>, <code>xgboost</code>
        </li>
    </ul>
    </p>
</div>

<a target="_blank" href="https://colab.research.google.com/github/kraina-ai/srai/blob/main/examples/use_cases/simple_machine_learning_with_overture_maps_data.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
# Uncomment the line below to install the required packages
# (e.g. when running in a new environment or Google Colab)

# %pip install srai[osm,overturemaps,plotting] contextily seaborn scikit-learn xgboost

## Import required libraries

In [ ]:
import contextily as cx
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import pandas as pd
import pyarrow as pa
import seaborn as sns
import xgboost as xgb
from h3 import int_to_str, str_to_int
from h3ronpy import grid_disk_aggregate_k
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
from sklearn.preprocessing import StandardScaler

from srai.embedders import ContextualCountEmbedder
from srai.h3 import h3_to_shapely_geometry
from srai.joiners import IntersectionJoiner
from srai.loaders import OSMOnlineLoader, OvertureMapsLoader
from srai.neighbourhoods import H3Neighbourhood
from srai.regionalizers import H3Regionalizer, geocode_to_region_gdf

In [ ]:
SEED = 71

## Define regions of interest

Geocode three Spanish cities using the [`geocode_to_region_gdf`](../../../api/regionalizers/#srai.regionalizers.geocode_to_region_gdf) function.

In [ ]:
cities_names = ["Madrid", "Seville", "Valencia"]
regions = geocode_to_region_gdf(cities_names)
regions.index = cities_names
regions

## Load bicycle-sharing stations data

Load locations of the bicycle-sharing station from OpenStreetMap using  [`OsmOnlineLoader`](../../loaders/osm_online_loader/) for the defined regions.

We will use the `{"amenity": "bicycle_rental"}` filter to get only locations where you can rental a bicycle (see the [`amenity=bicycle_rental`](https://wiki.openstreetmap.org/wiki/Tag:amenity%3Dbicycle_rental) definition on the OSM wiki).

In [ ]:
bicycle_stations = OSMOnlineLoader().load(area=regions, tags={"amenity": "bicycle_rental"})
bicycle_stations

Let's join the locations of the bicycle-sharing stations with defined cities geometries using [`IntersectionJoiner`](../../joiners/intersection_joiner/) to group them per city.

In [ ]:
bicycle_stations_in_city = IntersectionJoiner().transform(
    regions, bicycle_stations, return_geom=True
)

bicycle_stations_per_city = {}
for city_name in cities_names:
    bicycle_stations_per_city[city_name] = bicycle_stations_in_city.loc[city_name]

Let's visualise the Madrid data on the map.

In [ ]:
bicycle_stations_per_city["Madrid"].explore(tiles="CartoDB Positron")

## Regionalize points data into H3 and create regions dataset

Now we will transform the stations locations into the H3 cells using the [`H3Regionalizer`](../../regionalizers/h3_regionalizer/).

After that, we will buffer those cells with grid disks together with aggregating the minimal distance to the station for each buffered H3 cell. We will use the [`grid_disk_aggregate_k`](https://h3ronpy.readthedocs.io/en/latest/usage/grid.html#grid-disk-aggregates-with-h3ronpy-grid-disk-aggregate-k) function from the `h3ronpy` library.

Since `h3ronpy` operates on the Arrow tables and using integers as H3 indexes, we have to transform them from `str` to `int` and back again.

We have to define 3 variables:
- `H3_RESOLUTION`: What will be the H3 cell resolution?
- `H3_PREDICTION_RANGE`: What is the maximum distance we aim to predict?
- `H3_NEIGHBOURS`: How many neighbours do we want to have in the embedding context?

We will then buffer the generated cells up to the `H3_PREDICTION_RANGE` + `H3_NEIGHBOURS` distance.

In [ ]:
H3_RESOLUTION = 11
H3_PREDICTION_RANGE = 10
H3_NEIGHBOURS = 5


def buffer_h3_cells_with_aggregation(h3_regions):
    """Expand H3 regions and calculate minimal distance to origin cells."""
    return (
        pa.table(
            grid_disk_aggregate_k(
                h3_regions.index.map(str_to_int),
                H3_NEIGHBOURS + H3_PREDICTION_RANGE,
                "min",
            )
        )
        .to_pandas()
        .rename(columns={"k": "distance_to_station", "cell": "region_id"})
    )


h3_regionalizer = H3Regionalizer(resolution=H3_RESOLUTION)
h3_regions_gdfs = []
for city_name, bicycle_stations_data in bicycle_stations_per_city.items():
    city_h3_regions = h3_regionalizer.transform(bicycle_stations_data)

    expanded_city_h3_regions = buffer_h3_cells_with_aggregation(city_h3_regions)
    expanded_city_h3_regions["region_id"] = expanded_city_h3_regions["region_id"].map(int_to_str)
    expanded_city_h3_regions = expanded_city_h3_regions.set_index("region_id")
    expanded_city_h3_regions["city"] = city_name
    expanded_city_h3_regions = gpd.GeoDataFrame(
        expanded_city_h3_regions,
        geometry=h3_to_shapely_geometry(expanded_city_h3_regions.index),
        crs=4326,
    )
    h3_regions_gdfs.append(expanded_city_h3_regions)

h3_regions = gpd.pd.concat(h3_regions_gdfs)

h3_regions

Now, we'll visualise the generated H3 regions.

We will colour the cells based on the distance in the range defined in `H3_PREDICTION_RANGE` and we will colour the additional cells buffering the prediction range with additional neighbours (defined in the `H3_NEIGHBOURS` variable) in grey.

In [ ]:
temps_cmap = LinearSegmentedColormap.from_list(
    name="Temps",
    colors=[
        "#009392FF",
        "#39B185FF",
        "#9CCB86FF",
        "#E9E29CFF",
        "#EEB479FF",
        "#E88471FF",
        "#CF597EFF",
    ],
)

ax = h3_regions[
    (h3_regions.city == "Madrid") & (h3_regions.distance_to_station <= H3_PREDICTION_RANGE)
].plot(
    column="distance_to_station",
    figsize=(20, 20),
    cmap=temps_cmap,
    alpha=0.6,
    legend=True,
    legend_kwds={
        "shrink": 0.3,
        "location": "bottom",
        "label": "Distance to station",
        "pad": -0.05,
    },
)
h3_regions[
    (h3_regions.city == "Madrid") & (h3_regions.distance_to_station > H3_PREDICTION_RANGE)
].plot(ax=ax, color="gray", alpha=0.3)
bicycle_stations_per_city["Madrid"].representative_point().plot(ax=ax, color="black", markersize=1)

cx.add_basemap(ax, crs=h3_regions.crs, source=cx.providers.CartoDB.PositronNoLabels, zoom=13)
ax.set_axis_off()
ax.set_title("Distance to the nearest bike station in Madrid", fontsize=20)
plt.show()

## Prepare features for the model

Now we will download the geospatial data for all of the H3 cells and generate embeddings for the model.

### Download Overture Maps data

Now we will use [`OvertureMapsLoader`](../../loaders/overture_maps_loader/) to download the data for selected list of datasets defined by a theme/type pair. For each pair we will define a hierarchy depth used to aggregate features into columns.

Data is downloaded using the [`OvertureMaestro`](https://kraina-ai.github.io/overturemaestro/latest/examples/advanced_functions/wide_form/) library.

<div class="admonition info">
    <p class="admonition-title">Switch to features based on OpenStreetMap</p>
    <p>
     If you want to use OpenStreetMap data instead you can use the <a href="../../loaders/osm_pbf_loader/"><code>OSMPbfLoader</code></a> with <code>GEOFABRIK_LAYERS</code> filter.
    </p>
    <p>
    <div class="highlight"><pre><span></span><span class="kn">from</span> <span class="nn">srai.loaders.osm_loaders.filters</span> <span class="kn">import</span> <span class="n">GEOFABRIK_LAYERS</span>
<span class="kn">from</span> <span class="nn">srai.loaders.osm_loaders</span> <span class="kn">import</span> <span class="n">OSMPbfLoader</span>

<span class="n">features</span> <span class="o">=</span> <span class="n">OSMPbfLoader</span><span class="p">()</span><span class="o">.</span><span class="n">load</span><span class="p">(</span><span class="n">area</span><span class="o">=</span><span class="n">h3_regions</span><span class="p">,</span> <span class="n">tags</span><span class="o">=</span><span class="n">GEOFABRIK_LAYERS</span><span class="p">)</span>
</pre></div>
    </p>
</div>

In [ ]:
OVERTURE_MAPS_HIERARCHY_DEPTH_VALUES = {
    ("base", "infrastructure"): 1,
    ("base", "land"): 1,
    ("base", "land_use"): 1,
    ("base", "water"): 1,
    ("transportation", "segment"): 2,
    ("buildings", "building"): 2,
    ("places", "place"): 1,
}

features = OvertureMapsLoader(
    release="2024-12-18.0",
    theme_type_pairs=list(OVERTURE_MAPS_HIERARCHY_DEPTH_VALUES.keys()),
    hierarchy_depth=list(OVERTURE_MAPS_HIERARCHY_DEPTH_VALUES.values()),
    include_all_possible_columns=False,
).load(area=h3_regions)

features

### Intersect downloaded features with H3 regions

Now we will join geometries of each region and feature using [`STRTree`](https://geopandas.org/en/stable/docs/reference/sindex.html) spatial index from the `GeoPandas` library.

As a result, we will get a multiindex with `region_id` and `feature_id` columns.

In [ ]:
joint = IntersectionJoiner().transform(regions=h3_regions, features=features)
joint

### Create the embedding

The simplest possible embedding in geospatial context is the number of features per category in a given region. You can generate it using [`CountEmbedder`](../../embedders/count_embedder/) and it is the equivalent of [`CountVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) from the `scikit-learn` library.

Here we will use a slightly more advanced method, taking into account close proximity in geographical space, so that each region also has information about what is in its neighbourhood. We call it `Context` and we will use the [`ContextualCountEmbedder`](../../embedders/contextual_count_embedder/) to get this kind of embedding.

This method will retrieve information about neighbours at a distance less than or equal to the variable `H3_NEIGHBOURS` and add up the number of objects in each ring. Then those aggregated rings could be added as additional columns (`concatenate_vectors`=`True`) or added to existing columns using diminishing weight based on the distance from the central region (`concatenate_vectors`=`False`, default).

We also have to pass a `Neighbourhood` object, that will be used to calculate adjacency between regions. Because we are working with H3 cells, we can pass [`H3Neighbourhood`](../../neighbourhoods/h3_neighbourhood/) object to utilize fast internal `H3` methods used to calculate neighbourhood of an H3 cell, instead of relying on slower geometry-based operations.

Since the data from the `OvertureMapsLoader` is returned in the form of `bool` columns (instead of default `str` columns with `None` values), the have to change default value of the `count_subcategories` from `True` to `False`. All of the subcategories are already present in the final form of features `GeoDataFrame`.

<div class="admonition warning">
    <p class="admonition-title">Using OpenStreetMap features</p>
    <p>
     If you are using the OpenStreetMap data, then change the <code>count_subcategories</code> parameter to <code>True</code>.
    </p>
    <hr>
    <p>
     Also, remember to remove bicycle sharing stations from the dataset (<code>amenity=bicycle_sharing</code>), since we don't want it to be present in the final dataset.
    </p>
    <p>
     Overture Maps doesn't have any bicycle sharing stations data in the <code>2024-12-18.0</code> release, so we don't need to do this when using Overture Maps data.
    </p>
    <p>
    <div class="highlight"><pre><span></span><span class="n">embeddings</span> <span class="o">=</span> <span class="n">embeddings</span><span class="o">.</span><span class="n">drop</span><span class="p">(</span>
    <span class="n">columns</span><span class="o">=</span><span class="p">[</span>
        <span class="n">c</span> <span class="k">for</span> <span class="n">c</span> <span class="ow">in</span> <span class="n">embeddings</span><span class="o">.</span><span class="n">columns</span>
        <span class="k">if</span> <span class="s2">"bicycle_rental"</span> <span class="ow">in</span> <span class="n">c</span>
    <span class="p">]</span>
<span class="p">)</span>
</pre></div>
    </p>
</div>

In [ ]:
embeddings = ContextualCountEmbedder(
    neighbourhood=H3Neighbourhood(),
    neighbourhood_distance=H3_NEIGHBOURS,
    concatenate_vectors=False,
    count_subcategories=False,
).transform(regions_gdf=h3_regions, features_gdf=features, joint_gdf=joint)

embeddings

## Training the model

After preparing the features for all three cities at once, it is now time to split them, prepare them for training and train the `XGBoost` model.

### Define the target and cities for training

We will now define the target column that will be used for the model to predict and select train and validation cities.

In [ ]:
TARGET = "distance_to_station"
TRAIN_CITY = "Madrid"
VALIDATION_CITY = "Seville"

Let's join the cities datasets with generated embeddings.

In [ ]:
madrid_data = h3_regions[h3_regions["city"] == "Madrid"].merge(
    embeddings, left_index=True, right_index=True
)

seville_data = h3_regions[h3_regions["city"] == "Seville"].merge(
    embeddings, left_index=True, right_index=True
)

valencia_data = h3_regions[h3_regions["city"] == "Valencia"].merge(
    embeddings, left_index=True, right_index=True
)


madrid_data.shape, seville_data.shape, valencia_data.shape

In [ ]:
madrid_data.head()

### Scale the features

Each city might have a different density of features and we don't want to be sensitive to those changes. 

Let's consider a scenario where in a city A, there is a lot of benches, and a city B where there isn't a lot of benches. Now, there would naturally be a notable difference in the number of benches we can expect in a given region in a city A vs city B.
Scaling the features for each of those cities will give us a relative information about how many benches there are in a given region in regards to a particular city. And that's why we want to scale the features, to keep different cities in a similar domain, so that the transfer of knowledge between them is easier for the model. 

Here we will use a [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) from the `scikit-learn` library.

In [ ]:
x_madrid = StandardScaler().fit_transform(madrid_data[embeddings.columns])
y_madrid = madrid_data[TARGET]

x_seville = StandardScaler().fit_transform(seville_data[embeddings.columns])
y_seville = seville_data[TARGET]

x_valencia = StandardScaler().fit_transform(valencia_data[embeddings.columns])
y_valencia = valencia_data[TARGET]

x_madrid.shape, y_madrid.shape, x_seville.shape, y_seville.shape, x_valencia.shape, y_valencia.shape

### Train the model

It is finally time to train the model.

We will use the [`XGBoost`](https://xgboost.readthedocs.io/en/stable/) library as a popular choice in the data science domain.

We will create two datasets: train and validation and we will set some parameters for the booster training.

Our loss function will be Root Mean Squared Error (RMSE) and we will use early stopping to avoid overfitting to a single city.

In [ ]:
mask = y_madrid <= H3_PREDICTION_RANGE
dtrain = xgb.DMatrix(
    data=x_madrid[mask], label=y_madrid[mask], feature_names=list(embeddings.columns)
)
mask = y_seville <= H3_PREDICTION_RANGE
dval = xgb.DMatrix(
    data=x_seville[mask], label=y_seville[mask], feature_names=list(embeddings.columns)
)

# Set parameters for XGBoost
params = {
    "objective": "reg:squarederror",
    "eta": 0.01,
    "max_depth": 8,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "seed": SEED,
}

# Train the model
bst = xgb.train(
    params,
    dtrain,
    num_boost_round=2000,
    verbose_eval=50,
    early_stopping_rounds=100,
    evals=[(dtrain, "train"), (dval, "valid")],
)

In [ ]:
bst.best_iteration, bst.best_score

## Evaluate the results

Now we will evaluate the model and visualise the results on a map.

### Predict the distance for all cities

First we need to predict the distance for all three cities to have a set to compare to.

We will calculate the prediction error using predicted value and expected distance.

In [ ]:
concatenated_regions = pd.concat([madrid_data, seville_data, valencia_data])[
    [TARGET, "city", "geometry"]
]

concatenated_regions["predicted_distance_to_station"] = (
    np.concatenate(
        [
            bst.predict(xgb.DMatrix(x_madrid, feature_names=list(embeddings.columns))),
            bst.predict(xgb.DMatrix(x_seville, feature_names=list(embeddings.columns))),
            bst.predict(xgb.DMatrix(x_valencia, feature_names=list(embeddings.columns))),
        ]
    )
    .round(2)
    .clip(min=0)
)

concatenated_regions = concatenated_regions[concatenated_regions[TARGET] <= H3_PREDICTION_RANGE]

concatenated_regions

### Visualise the distribution of predictions

First, we will see how the model performed in terms of distance prediction by comparing the predicted values with the expected values.

In [ ]:
pal12_cmap = ListedColormap(
    name="pal12",
    colors=[
        "#DE1A1AFF",
        "#E1581AFF",
        "#E37F1BFF",
        "#E2A11BFF",
        "#DFC11BFF",
        "#D8E01BFF",
        "#CEFF1AFF",
        "#B2D736FF",
        "#97B043FF",
        "#7B8948FF",
        "#5F654AFF",
        "#43444AFF",
        "#202547FF",
    ][: (H3_PREDICTION_RANGE + 1)],
)

_, axs = plt.subplots(2, 3, figsize=(12, 8), sharey=True, sharex=True, dpi=600)

axs[0, 0].set_ylabel("Predicted distance to station")
axs[1, 0].set_ylabel("Predicted distance to station")

for idx, city_name in enumerate(cities_names):
    expected_values = concatenated_regions[concatenated_regions["city"] == city_name][TARGET]
    mask = expected_values <= H3_PREDICTION_RANGE
    expected_values = expected_values[mask]
    predicted_values = concatenated_regions[concatenated_regions["city"] == city_name][
        "predicted_distance_to_station"
    ][mask]

    sns.regplot(
        x=expected_values,
        y=predicted_values,
        ax=axs[0, idx],
        scatter=True,
        order=2,
        scatter_kws=dict(
            alpha=0.02,
            color=[pal12_cmap.colors[_y] for _y in expected_values],
        ),
        line_kws=dict(
            color="black",
        ),
        x_jitter=0.1,
    )
    sns.violinplot(
        x=expected_values,
        y=predicted_values,
        ax=axs[1, idx],
        fill=True,
        palette=pal12_cmap.colors,
        hue=expected_values,
        legend=False,
    )
    title = city_name
    if city_name == TRAIN_CITY:
        title += " (train)"
    elif city_name == VALIDATION_CITY:
        title += " (validation)"

    axs[0, idx].set_title(title)
    axs[0, idx].set_xlabel(None)
    axs[1, idx].set_xlabel("Distance to station")

axs[0, 0].set_ylim(bottom=0)
axs[1, 0].set_ylim(bottom=0)

plt.tight_layout()

As you can see, it's far from perfect, but we can see that the overall trend from the lowest to the highest distance is preserved for both validation city (Seville) and the city that didn't take part in the modelling (Valencia).

Although the scale isn't preserved and errors are significant, we can still use those predictions to display some hotspots on a map.

### Visualise the predictions on a map

Here comes the most interesting part - visualising the predictions on a map.

We will change the scale for each of the cities to mach the scale of predictions, so that we can clearly see the difference between lowest and highest predictions.

Existing bicycle stations will be plotted as black dots.

In [ ]:
for city_name in cities_names:
    city_data = concatenated_regions[concatenated_regions.city == city_name]
    ax = city_data.plot(
        column="predicted_distance_to_station",
        figsize=(20, 20),
        cmap=temps_cmap,
        alpha=0.8,
        legend=True,
        legend_kwds={
            "shrink": 0.3,
            "location": "bottom",
            "label": "Predicted distance to station",
            "pad": -0.05,
        },
        vmin=max(0, city_data["predicted_distance_to_station"].min()),
        vmax=city_data["predicted_distance_to_station"].max(),
    )
    bicycle_stations_per_city[city_name].representative_point().plot(
        ax=ax, color="black", markersize=3, alpha=0.4
    )

    cx.add_basemap(ax, crs=h3_regions.crs, source=cx.providers.CartoDB.PositronNoLabels, zoom=13)
    ax.set_axis_off()

    title = f"Predicted distance to the nearest bike station in {city_name}"
    if city_name == TRAIN_CITY:
        title += " (train)"
    elif city_name == VALIDATION_CITY:
        title += " (validation)"

    ax.set_title(title, fontsize=20)

    plt.show()

### Visualise the predictions error on a map

And the last part, the prediction error preview.

As in other areas of data science, we can make a boring error graph on the X / Y axis or as a histogram. But what would the field of geospatial data be if we didn't also use maps for this.

Since we already know that ranges in the predictions per city aren't matched with the original domain, we will rescale the predictions to be in the original domain and recalculate the prediction error. That way we will be able to eliminate the base error resulting from the difference between scales and focus on the spatial part of the error.

We will also normalize the prediction error, to the 0-1 scale so that the biggest negative error will correlate with the value of 0, biggest positive error to the value of 1, and value of 0.5 will represent no error at all.

That way we will have an easy way to utilize divergent colour scale without rescaling it.

Additionally, we will use opacity to hide regions where the error was insignificant.

In [ ]:
tangerine_blues_cmap = LinearSegmentedColormap.from_list(
    name="TangerineBlues",
    colors=[
        "#552000FF",
        "#8A4D00FF",
        "#C17D17FF",
        "#F8B150FF",
        "#F5F5F5FF",
        "#93C6E1FF",
        "#5F93ACFF",
        "#2E627AFF",
        "#00344AFF",
    ],
)

for city_name in cities_names:
    city_data = concatenated_regions[concatenated_regions.city == city_name].copy()

    # NewValue = (((OldValue - OldMin) * (NewMax - NewMin)) / (OldMax - OldMin)) + NewMin
    city_data["scaled_predicted_distance_to_station"] = (
        (
            (
                city_data["predicted_distance_to_station"]
                - city_data["predicted_distance_to_station"].min()
            )
            * (city_data["distance_to_station"].max() - city_data["distance_to_station"].min())
        )
        / (
            city_data["predicted_distance_to_station"].max()
            - city_data["predicted_distance_to_station"].min()
        )
    ) + city_data["distance_to_station"].min()

    city_data["scaled_prediction_error"] = (
        city_data[TARGET].clip(upper=H3_PREDICTION_RANGE)
        - city_data["scaled_predicted_distance_to_station"]
    )

    city_data["normalized_scaled_prediction_error"] = (
        city_data["scaled_prediction_error"].apply(
            lambda x, city_data=city_data: (
                -x / city_data["scaled_prediction_error"].min()
                if x < 0
                else x / city_data["scaled_prediction_error"].max()
            )
        )
        + 1
    ) / 2

    city_data["normalized_prediction_error_alpha"] = (
        city_data["normalized_scaled_prediction_error"] - 0.5
    ).abs() * 2

    ax = city_data.plot(
        column="normalized_scaled_prediction_error",
        figsize=(20, 20),
        cmap=tangerine_blues_cmap,
        alpha=city_data["normalized_prediction_error_alpha"],
        legend=True,
        legend_kwds={
            "shrink": 0.3,
            "location": "bottom",
            "label": "Distance prediction error (scaled)",
            "pad": -0.05,
            "ticks": [0, 0.5, 1],
            "format": mticker.FixedFormatter(
                [
                    round(city_data["scaled_prediction_error"].min(), 2),
                    "0",
                    round(city_data["scaled_prediction_error"].max(), 2),
                ]
            ),
        },
    )
    bicycle_stations_per_city[city_name].representative_point().plot(
        ax=ax, color="black", markersize=3, alpha=0.4
    )

    cx.add_basemap(ax, crs=h3_regions.crs, source=cx.providers.CartoDB.PositronNoLabels, zoom=13)
    ax.set_axis_off()

    title = f"Distance prediction error (scaled) in {city_name}"
    if city_name == TRAIN_CITY:
        title += " (train)"
    elif city_name == VALIDATION_CITY:
        title += " (validation)"

    ax.set_title(title, fontsize=20)

    plt.show()

Below is an example of the scaling we have done in a previous step.

The difference between predicted distances and predicted distances scaled to the original domain. 

In [ ]:
sns.displot(
    data=pd.melt(
        city_data,
        value_vars=[
            "predicted_distance_to_station",
            "scaled_predicted_distance_to_station",
        ],
    ),
    x="value",
    hue="variable",
    kde=True,
    height=10,
)

### See feature importance 

We can visualise feature importance from the `XGBoost` Booster object and display the gain per feature. This info can tell us, or the planners, what urban features are important for the model. 

This is a basic form of explainability and it doesn't tell us if the feature has negative or positive impact on the result. For that we would have to use for example the SHAP library. 

In [ ]:
_, ax = plt.subplots(1, 1, figsize=(10, 10))

feature_importance_df = pd.DataFrame(
    bst.get_score(importance_type="gain").items(), columns=["Feature", "Gain"]
)
ax = sns.barplot(y="Feature", x="Gain", data=feature_importance_df.nlargest(20, columns="Gain"))
ax.set_title("Feature importance")
plt.show()

## Summary

In summary, looking at the prediction maps, and the errors, there is a general trend where the predictions fairly represent the true layout of the cycle stations.

There is a strong predominance of city centres, where these predictions are closer to reality, and the model tends to predict low values there. As you move away from the centre, you can see increasing deviations, with noticeable clusters where the model completely failed to predict the existence of a cycle station.

This may be due to a number of factors:
- differences in planning of the cycling network between towns and cities;
- differences in urban layout of towns and cities;
- lack of features or examples representing some types of urban tissue;
- model unable to catch all the complexities of the data set;
- too simple method of embedding geographic features into the numerical vector.

Even looking at the example of Madrid, which has been reproduced quite well by the model, you can see examples where the model thinks a station should not be there, or has found empty places where it would propose to put a station, so even without knowledge transfer between cities, such a model could support local planners.

---

I hope that the methods presented here have helped you understand the basics of the field of geospatial machine learning and presented the tangible benefits of using such a model in real-world scenarios.